# Histogram Model: Color and Position Features
----
Sergei Papulin (papulin.edu@gmail.com)

## Contents

- [Generating Images](#Generating-Images)
- [Defining Color Elements](#Defining-Color-Elements)
- [Defining Positional Elements](#Defining-Positional-Elements)
- [Creating Histogram](#Creating-Histogram)
- [Querying](#Querying)
- [Image Retrieval](#Image-Retrieval)
- [References](#References)

### Creating virtual environment

This is an optional step. You can skip it and install packages to your current environment.

```bash
python -m venv .venv/histtest
source .venv/histtest/bin/activate
pip install \
    numpy==1.19.5 \
    plotly==5.5.0 \
    jupyter==1.0.0 \
    pillow==5.4.1 \
    scikit-image==0.14.2 \
    pycocotools==2.0.3 \
    himpy=0.0.1
```

#### Load packages

In [ ]:
import sys
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
from himpy.histogram import operations
from himpy.executor import Parser, Evaluator
from himpy.utils import E

In [ ]:
%load_ext autoreload
%autoreload 2

# dataset generator
from utils.datasets import ColorImageGenerator

# feature extraction
from utils.feature_extraction import (
    FeatureMerger,
    ColorSetTransformer,
    PositionSetTransformer,
    filter_data,
    create_histogram,
    create_histogram_,
    extract_elements,
    extract_element_set
)

# search engine
from utils.search_engine import SearchEngine

# plot
from utils.plot.plotly_plot import (
    show_color_elements,
    plot_position_grid_plotly,
    plot_histogram,
    show_rank_images
)


# image colors
from utils.feature_extraction.color import COLOR_ELEMENTS, COLOR_ELEMENTS_RGB

## Generating Images

In [ ]:
image_generator = ColorImageGenerator()

In [ ]:
image = image_generator.generate(shape=(100, 100), 
                                 steps=(10, 10), 
                                 normal_element_ids={"e7", "e9", "e34"},
                                 random_state=123)

In [ ]:
fig = make_subplots(rows=1, cols=1, subplot_titles=("Initial Image [RGB]",))

fig.add_trace(go.Image(z=image, hoverinfo="skip"), row=1, col=1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=250, height=200)
fig.show()

## Defining Color Elements

### Low-Level Elements

Stucture of color elements:

```json
[
    {
        "id": id,
        "h": [min, max],
        "s": [min, max],
        "b": [min, max]
    }, ...
]
```

Parameters:

- `id`: identification of an element
- `h`: hue, interval. If you define interval from 230 to 10, it will be divided into two ones: from 230 to 240 and from 0 to 10
- `s`: saturation, interval
- `b`: brightness, interval


In [ ]:
# Show the first five low-level color elements
print("Total number of color elements:", len(COLOR_ELEMENTS))
COLOR_ELEMENTS[:5]

In [ ]:
# Display all low-level color elements
show_color_elements(title="Low-level elements (Color Universal set - Uc)")

In [ ]:
# Display all low-level color elements
show_color_elements(element_ids={"e18"}, title="e18")

In [ ]:
color_transformer = ColorSetTransformer()
color_image = color_transformer.fit_transform(X=image, y=None)
color_image

In [ ]:
color_image_ = color_transformer.transform_to_int(color_image)


fig = make_subplots(rows=1, cols=2, subplot_titles=("Initial Image [RGB]", "Transformed Image [low-level]"))

common_heatmap_args = {
    "zmax": 40,
    "zauto": False,
    "colorscale": "gray",
    "showscale": False,
    "hoverinfo": "skip"
}

fig.add_trace(go.Image(z=image, hoverinfo="skip"), row=1, col=1)
fig.add_trace(go.Heatmap(z=color_image_, **common_heatmap_args), row=1, col=2)
fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", row=1, col=2)
fig.update_xaxes(constrain="domain", scaleanchor="y", row=1, col=2)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=500, height=200)
fig.show()

### High-Level Elements

In [ ]:
parser = Parser()

In [ ]:
# Definition of high-level positional elements

Ec_green        = E("e1+e2+e3+e4+e5+e6+e7+e8+e9+e10+e11+e12+e13+e14+e15+e16+e17+e18+e19+e20")
Ec_yellow_green = E("e2+e3+e21+e22+e23+e24+e25+e26+e27+e28+e29+e30")
Ec_red          = E("e31+e32+e33+e34+e35+e36+e37+e38+e39+e40")
Ec_rose         = E("e32+e35+e36+e39+e40")


Ecs = [
    ("green", Ec_green),
    ("yellow_green", Ec_yellow_green),
    ("red", Ec_red),
    ("rose", Ec_rose)
]


# Sets of hight-level color elements (they will be used for the Evaluator below)

Ecs_set = { name: parser.parse_set(Ec.value) for name, Ec in Ecs}
Ecs_set["green"]

In [ ]:
# Display all low-level color elements
for title, Ec_set in Ecs_set.items():
    show_color_elements(element_ids=Ec_set, title=title)

In [ ]:
color_filtered_image      = color_transformer.filter_elements(color_image, Ecs_set["green"])
color_data_filtered_image = color_transformer.filter_data(color_image, image, Ecs_set["green"])

In [ ]:
color_filtered_image_ = color_transformer.transform_to_int(color_filtered_image)

In [ ]:
fig = make_subplots(rows=1, cols=4, 
                    subplot_titles=(
                        "Initial Image [RGB]", 
                        "Transformed Image [low-level]", 
                        "Element Filter [green]", 
                        "Data Filter [green]"
                    ))

common_heatmap_args = {
    "zmax": 40,
    "zauto": False,
    "colorscale": "gray",
    "showscale": False,
    "hoverinfo": "skip"
}

fig.add_trace(go.Image(z=image, hoverinfo="skip"), row=1, col=1)
fig.add_trace(go.Heatmap(z=color_image_, **common_heatmap_args), row=1, col=2)
fig.add_trace(go.Heatmap(z=color_filtered_image_, **common_heatmap_args), row=1, col=3)
fig.add_trace(go.Image(z=color_data_filtered_image, hoverinfo="skip"), row=1, col=4)
fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", row=1, col=2)
fig.update_xaxes(constrain="domain", scaleanchor="y", row=1, col=2)
fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", row=1, col=3)
fig.update_xaxes(constrain="domain", scaleanchor="y", row=1, col=3)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=800, height=200)
fig.show()

## Defining Positional Elements

### Low-Level Elements

In [ ]:
# Grid params: 5 splits along Y, and 5 along X
GRID = (5, 5)

# Create a position transformer
position_transformer = PositionSetTransformer(splits=GRID, element_ndim=3)

# Set an image size
position_transformer.fit(X=image, y=None)

# Build an image in which each pixel defines a position
position_image = position_transformer.transform(X=image)

In [ ]:
fig = make_subplots(rows=1, cols=1, subplot_titles=("Position Elements",))
fig = plot_position_grid_plotly(position_transformer, fig, row=1, col=1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=300, height=200)
fig.show()

In [ ]:
color_image_ = color_transformer.transform_to_int(color_image)


fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=(
                        "Initial Image [RGB]", 
                        "Transformed Image [low-level]", 
                        "Position Elements"
                    ))

common_heatmap_args = {
    "zauto": True,
    "colorscale": "gray",
    "showscale": False,
    "hoverinfo": "skip"
}

fig.add_trace(go.Image(z=image, hoverinfo="skip"), row=1, col=1)
fig.add_trace(go.Heatmap(z=position_image, **common_heatmap_args), row=1, col=2)
fig = plot_position_grid_plotly(position_transformer, fig, row=1, col=3)
fig.update_yaxes(autorange="reversed", constrain="domain", scaleanchor="x", row=1, col=2)
fig.update_xaxes(constrain="domain", scaleanchor="y", row=1, col=2)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=900, height=200)
fig.show()

### High-Level Elements

In [ ]:
# Definition of high-level positional elements

Ep_top    = E("1+2+3+4+5+6+7+8+9+10")
Ep_bottom = E("16+17+18+19+20+21+22+23+24+25")
Ep_left   = E("1+2+6+7+11+12+16+17+21+22")
Ep_right  = E("4+5+9+10+14+15+19+20+24+25")
Ep_center = E("7+8+9+12+13+14+17+18+19")

Eps = [
    ("top", Ep_top), 
    ("bottom", Ep_bottom), 
    ("left", Ep_left), 
    ("right", Ep_right), 
    ("center", Ep_center)
]


# Sets of high-level positional elements (they will be used for the Evaluator below)

Eps_set = { name: parser.parse_set(Ep.value) for name, Ep in Eps}
Eps_set["center"]

In [ ]:
fig = make_subplots(rows=1, cols=len(Eps_set), subplot_titles=tuple(Eps_set.keys()))
for i, (name, Eps) in enumerate(Eps_set.items()):
    fig = plot_position_grid_plotly(position_transformer, fig, element_ids=Eps, row=1, col=i+1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=len(Eps_set)*150, height=200)
fig.show()

In [ ]:
Ep_set_ = Eps_set["center"]

elements_image = position_transformer.filter_elements(position_image, Ep_set_)
filtered_image = position_transformer.filter_data(position_image, image, Ep_set_)

# TODO: plot

In [ ]:
# Note: Plotly plot images from bottom left corner. So, images are inversed. 
#  [::-1,:,:] is used to recover the origin

fig = make_subplots(rows=1, cols=len(Eps_set), subplot_titles=tuple(Eps_set.keys()))
for i, (name, Ep_set) in enumerate(Eps_set.items()):
    filtered_image = position_transformer.filter_data(position_image, image, Ep_set)
    fig.add_trace(go.Image(z=filtered_image[::-1,:,:], hoverinfo="skip"), row=1, col=i+1)
    fig = plot_position_grid_plotly(position_transformer, fig, element_ids=Ep_set, row=1, col=i+1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=len(Eps_set)*150, height=200)
fig.show()

## Creating Histogram

In [ ]:
# Option 1
hist = create_histogram((position_image, color_image))
hist.to_dict()

In [ ]:
# Option 2.a
feature_merger = FeatureMerger()
merged_image = feature_merger.fit_transform((position_image, color_image))
merged_image

In [ ]:
# Option 2.b
hist = create_histogram_(merged_image)
hist.to_dict()

In [ ]:
hist_elements = sorted(hist.hist_elements().items(), key=lambda x: int(x[0][1].lstrip("e")))
elements = ["({})".format(",".join(el[0])) for el in hist_elements]
values = [el[1].value for el in hist_elements]
colors = ["rgb{}".format(COLOR_ELEMENTS_RGB[el[0][1]]) for el in hist_elements]


fig = make_subplots(rows=1, cols=2, column_widths=[0.2, 0.8], subplot_titles=("Image", "Histogram"))

fig.add_image(z=image, row=1, col=1, name="image")
fig.add_bar(x=elements, y=values, marker_color=colors, width=0.5, row=1, col=2, name="histogram")

fig.update_xaxes(gridcolor='#bdbdbd', title="Elements", titlefont=dict(color="grey"), row=1, col=2)
fig.update_yaxes(gridcolor='#bdbdbd', title="Counts", titlefont=dict(color="grey"), row=1, col=2)

fig.update_layout(plot_bgcolor='#fefefe', showlegend=False, height=300, width=900, title_text="Initial Data")
fig.show()

## Querying

In [ ]:
high_level_elements = {
    # position
    0: Eps_set, 
    # color
    1: Ecs_set
}

In [ ]:
# Initialize an evaluator
evaluator = Evaluator(operations, hist, high_level_elements=high_level_elements)

In [ ]:
E1 = E("top", "green")
E2 = E("right", "red")

In [ ]:
E1_expr = parser.parse_string(E1.value)
HE1 = evaluator.eval(E1_expr)
print("Expression for E1:\n{}".format(E1.value))
print("\nThe parsed expressino for E1 in the postfix notation:\n{}".format(E1_expr))
print("\nHistogram of E1 given the image:\n{}".format(HE1.to_dict()))
print("\nValue of presence for E1:\n{}".format(HE1.sum()))

In [ ]:
E2_expr = parser.parse_string(E2.value)
HE2 = evaluator.eval(E2_expr)
print("Expression for E2:\n{}".format(E2.value))
print("\nThe parsed expressino for E2 in the postfix notation:\n{}".format(E2_expr))
print("\nHistogram of E2 given the image:\n{}".format(HE2.to_dict()))
print("\nValue of presence for E2:\n{}".format(HE2.sum()))

In [ ]:
def show_histogram_elements(image, merged_image, HE, title):
    E_set = extract_element_set(HE, 2)
    E_image = filter_data(image, merged_image, HE.elements())
    
    hist_elements = sorted(HE.to_dict().items(), key=lambda x: int(x[0][1].lstrip("e")))
    elements = ["({})".format(",".join(el[0])) for el in hist_elements]
    values = [el[1] for el in hist_elements]
    colors = ["rgb{}".format(COLOR_ELEMENTS_RGB[el[0][1]]) for el in hist_elements]
    
    fig = make_subplots(rows=1, cols=2, column_widths=[0.2, 0.8], 
                    subplot_titles=("Image<br> {}".format(title), "Histogram: {}".format(title)))
    fig.add_image(z=E_image, row=1, col=1, name="image")
    fig = plot_histogram(elements=elements, values=values, colors=colors, fig=fig, row=1, col=2)

    image_axes_args = {
        "showticklabels": False,
        "showline": True,
        "linewidth": 1,
        "linecolor": "black",
        "mirror": True
    }
    
    fig.update_xaxes(row=1, col=1, **image_axes_args)
    fig.update_yaxes(row=1, col=1, **image_axes_args)

    fig.update_layout(
        margin=dict(l=40, r=40, t=40, b=20), 
        plot_bgcolor="#fefefe", showlegend=False, 
        height=300, width=900)
    fig.show()

In [ ]:
show_histogram_elements(image, merged_image, HE1, E1.value)
show_histogram_elements(image, merged_image, HE2, E2.value)

## Operations on Histogram Elements

### Example for Union

In [ ]:
# Expression with union
E_union = E1 + E2

# Parsed expression
E_union_expr = parser.parse_string(E_union.value)

# Calculate histogram value
HE_union = evaluator.eval(E_union_expr)

print("Expression for E_union:\n{}".format(E_union))
print("\nThe parsed expression for E_union in the postfix notation:\n{}".format(E_union_expr))
print("\nHistogram of E_union given the image:\n{}".format(HE_union.to_dict()))
print("\nValue of presence for E_union:\n{}".format(HE_union.sum()))

In [ ]:
show_histogram_elements(image, merged_image, HE1, E1.value)
show_histogram_elements(image, merged_image, HE2, E2.value)
show_histogram_elements(image, merged_image, HE_union, E_union.value)

### Other operations

In [ ]:
operation_list = [
    # set operations
    ("union",          "+",    E1 + E2), 
    ("intersection",   "*",    E1 * E2),
    ("substraction",   "-",    E1 - E2),  # or exception, or E1.Sub(E2)
    # logic operations
    ("and",            "&",    E1 & E2),  # or E1.And(E2)
    ("or",             "|",    E1 | E2),  # or E1.Or(E2)
    ("xor",            "^",    E1 ^ E2),  # or E1.Xor(E2)
    ("xsubstraction",  "Xsub", E1.Xsub(E2)),
]

In [ ]:
for op_name, op_sign, op in operation_list:
    E_expr = parser.parse_string(op.value)
    HE = evaluator.eval(E_expr)
    print("{:12}{:^12}{:10}".format("Operation", "Sign", "Result"))
    print("{}".format("-"*34))
    print("{:12}{:^12}{:.5f}".format(op_name, op_sign, HE.sum()))
    show_histogram_elements(image, merged_image, HE1, E1.value)
    show_histogram_elements(image, merged_image, HE2, E2.value)
    show_histogram_elements(image, merged_image, HE, op.value)

## Image Retrieval

### Expression as query

In [ ]:
# Images with normal distrubited some elements 
images = [
    image_generator.generate(
        shape=(100, 100), 
        steps=(10, 10), 
        normal_element_ids={"e33", "e34"},
        random_state=i+100) 
    for i in range(100)
]


# Images with uniform distributed elements
images += [
    image_generator.generate(
        shape=(100, 100), 
        steps=(10, 10)) 
    for i in range(100)    
]

In [ ]:
# Create histograms for the images
hists = list()
limit = len(images)

position_image = position_transformer.fit_transform(X=images[0], y=None)

for indx, image in enumerate(images):
#     position_image = position_transformer.fit_transform(shape=image.shape[:-1])
    color_image = color_transformer.fit_transform(X=image, y=None)
    hist = create_histogram((position_image, color_image))
    hists.append((indx, hist))
    print("\rCurrent image index: {}/{}".format(indx + 1, limit), end="")

In [ ]:
# Initialize a search engine
search_engine = SearchEngine(hists, parser, evaluator)

In [ ]:
TOP_N = 20

In [ ]:
# Elements
E1 = E("top", "green")
E2 = E("right", "red")

# TODO: show E1 = E("*", "green")

# Define your query
query = E1 & E2

# Retrieve images using the query
ranked_images = search_engine.retrieve(query, topN=TOP_N)
print("Total retrieved images:", len(ranked_images))
ranked_images[:5]

In [ ]:
fig = show_rank_images(images, ranked_images, limit=TOP_N, 
                       title="Top {}: <b>{}</b>".format(TOP_N, query.value))
fig.show()

### Image sample as query

In [ ]:
# Generate a new sample image
sample_image = image_generator.generate(
    shape=(100, 100), 
    steps=(10, 10), 
    normal_element_ids={"e33", "e34"},
    random_state=1) 

In [ ]:
fig = make_subplots(rows=1, cols=1, subplot_titles=("Sample Image",))

fig.add_trace(go.Image(z=sample_image, hoverinfo="skip"), row=1, col=1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=200, height=200)
fig.show()

In [ ]:
# Transform the image to histogram
position_image = position_transformer.fit_transform(X=sample_image)
color_image = color_transformer.transform(sample_image)
sample_hist = create_histogram((position_image, color_image))

In [ ]:
# Retrieve images similar to the sample
ranked_images__sample = search_engine.retrieve(sample_hist, topN=TOP_N)
print("Total retrieved images:", len(ranked_images__sample))
ranked_images__sample[:5]

In [ ]:
fig = make_subplots(rows=1, cols=1, subplot_titles=("Sample Image",))

fig.add_trace(go.Image(z=sample_image, hoverinfo="skip"), row=1, col=1)
fig.update_yaxes(showticklabels=False)
fig.update_xaxes(showticklabels=False)
fig.update_layout(margin=dict(l=20, r=20, t=40, b=20), width=200, height=200)
fig.show()

fig = show_rank_images(images, ranked_images__sample, 
                       limit=TOP_N, title="Top {}: <b>Sample Image</b>".format(TOP_N))
fig.show()

## References

- Papulin S. [Introduction to Histogram Model](https://htmlpreview.github.io/?https://github.com/LSHist/histogram/blob/master/docs/hm_basics.html)
- Papulin S. [Multidimensional Histogram Model](https://htmlpreview.github.io/?https://github.com/LSHist/histogram/blob/master/docs/hm_multidim.html)